# IDE

In [1]:
# requriments
import pandas as pd

from sklearn.model_selection import train_test_split

from sklearn.ensemble import ExtraTreesRegressor

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

URL = "/kaggle/input/concrete-strength-regression/train.csv"
URL_SAMPLE = "/kaggle/input/concrete-strength-regression/sample_submission.csv"
URL_TEST = "/kaggle/input/concrete-strength-regression/test.csv"

TARGET = "csMPa"

# carregar dados
data = pd.read_csv(URL)

# divdir dados
X, y = data.drop(TARGET, axis=1), data[TARGET]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42
)
# modelos
baseline = ExtraTreesRegressor(random_state=42)

params = {  # some simple parameters to grid search
    "extratreesregressor__max_depth": [10, None],
    "extratreesregressor__n_estimators": [10, 50, 100, 500],
    "extratreesregressor__criterion": ["squared_error", "friedman_mse"],
}


# pipeline
pipeline = make_pipeline(StandardScaler(), baseline)

# cross-validation
kf = KFold(n_splits=10)
grid = GridSearchCV(estimator=pipeline, param_grid=params, cv=kf)

In [2]:
# treinar
grid.fit(X_train, y_train)

GridSearchCV(cv=KFold(n_splits=10, random_state=None, shuffle=False),
             estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                       ('extratreesregressor',
                                        ExtraTreesRegressor(random_state=42))]),
             param_grid={'extratreesregressor__criterion': ['squared_error',
                                                            'friedman_mse'],
                         'extratreesregressor__max_depth': [10, None],
                         'extratreesregressor__n_estimators': [10, 50, 100,
                                                               500]})

In [3]:
grid.best_params_

{'extratreesregressor__criterion': 'squared_error',
 'extratreesregressor__max_depth': None,
 'extratreesregressor__n_estimators': 100}

# Display

## Evaluation

In [4]:
y_hat = grid.predict(X_test)
eval_test = mean_squared_error(y_test, y_hat)
print(f"On test set: {eval_test:.2f}")

On test set: 15.39


In [5]:
y_hat_train = grid.predict(X_train)
eval_train = mean_squared_error(y_train, y_hat_train)
print(f"On train set: {eval_train:.2f}")

On train set: 0.00


# Submission

In [6]:
teste = pd.read_csv(URL_TEST)
sample = teste

sample = sample[["Row ID"]]
sample[TARGET] = grid.predict(teste)
sample.to_csv("submission.csv", index=False)

/tmp/ipykernel_18/1462406833.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample[TARGET] = grid.predict(teste)
